# Clustering Tutorial

## What's in scope
- what is clustering?
- clustering algorithms
    - k-means
    - hierarchical clustering
        - affinity propagation
- clustering word embeddings
    - how to get insight from clusters of word embeddings
- labeling clusters using a hierarchy

## What's not in scope?
- any other clustering algorithm
- any other unsupervised learning algorithm

# What is Clustering?

Remember how we had feature vectors (or word embeddings) that lived in n-dimensional space? Clustering is an [unsupervised learning algorithm](https://en.wikipedia.org/wiki/Unsupervised_learning) that puts similar feature vectors into the same group or cluster. 

![hierarchical clustering](figures/hierarchical-clustering.gif)

## When should I use clustering?

If you're wondering "what don't I know about this data set?", clustering will give some insight. 

Clustering can also be used as a feature extraction piece in the predictive modeling process.

In [1]:
# first, let's load in the FastText model from before so we're ready to cluster them
import os

model_file = os.path.join('models', 'phrased_got_ft.model')


In [3]:
# in all of these clustering trials, we'll need a way to link
# the word vector from FastText to the word itself. 

"""
---------------------------------------------------------------
Objective: Keep track of your words and vectors in a smart way
---------------------------------------------------------------
each clustering algorithm will need the input as an iterable of 
vectors (and nothing else)

it'll return a cluster model object, which has an attribute model.labels_

These labels will be an index of the cluster this vector fell into

But we want to know what _word_ fell into each cluster

So we need a map to know which vector corrresponds to each word

You might want a few jupyter notebook cells to explore 
what the FastText model has (maybe a vocab attribute?)
"""

In [4]:
# we're going to do this a few times, so let's make a function

def get_vectors_and_words(model):
    pass

vectors, words = get_vectors_and_words(model)

## Clustering Algorithms

### k-means
![k-means](figures/K-means-convergence.gif)

As an input, the k-means algorithm requires the number of clusters present `k`, and a maximum number of iterations `i`.

This algorithms consists of alternating two steps `i` times
1. assignment step: Assign each observation to the cluster whose mean has the least squared Euclidean distance, this is intuitively the "nearest" mean.
2. update step: Calculate the new means (centroids) of the observations in the new clusters. Do this by averaging the vectors for all instances in a given cluster

Quiz: What's the run time for k-means for `n` data points?

In [5]:
# let's implement k-means on our word data
from sklearn.cluster import KMeans


In [7]:
# we want the attribute of the KMeans model model.labels_


In [8]:
'''
-----------------------------------------------------------------
Objective: find a way to view words that are in the same cluster
-----------------------------------------------------------------
Right now we have the words ['these', 'are', 'unique', 'vocab', 'items', ...]
and the labels [0, 1, 1, 0, 2, ...]
These labels don't mean anything except that all the 1s go together, all the 2s, and so on

Figure out a way to hold this information so you can see which words are in which clusters
'''

def get_clusters(words, cluster_model):
    pass

In [9]:
# show the first few clusters by printing them


#### How did K-means do?

How do the clusters look? Do they make sense? 

How do we know that our number of clusters guess was right?


### Hierarchical Clustering

[Hierarchical clustering](https://en.wikipedia.org/wiki/Hierarchical_clustering) is a method of cluster analysis which seeks to build a hierarchy of clusters. There are two ways to do this

1. Agglomerative: bottom-up. Each observation starts in its own cluster and pairs of clusters are merged as we work up the hierarchy
2. Divisive: top-down. All observations start in one cluster, and splits are performed recursively as we move down the hierarchy.


### Affinity Propagation - a type of Hierarchical Clustering
![affinity propagation](figures/affinity-propagation.gif)

gif taken from [NIPS 2017 paper video](https://www.youtube.com/watch?v=1IOEFNGPNJc)

From [sklearn's documentation](https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation):
>AffinityPropagation creates clusters by sending messages between pairs of samples until convergence. A dataset is then described using a small number of exemplars, which are identified as those most representative of other samples. The messages sent between pairs represent the suitability for one sample to be the exemplar of the other, which is updated in response to the values from other pairs. This updating happens iteratively until convergence, at which point the final exemplars are chosen, and hence the final clustering is given.

One of the major benefits of affinity propagation is that it determines the number of clusters automatically. 

One of the downfalls of affinity propagation is the run time: `O(in^2)`

For more parameter's that come in sklearn, check [the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html#sklearn.cluster.AffinityPropagation).

In [10]:
'''
---------------------------------------
objective: cluster the word embeddings
---------------------------------------
we're going to use sklearn's affinity propagation 

'''

from sklearn.cluster import AffinityPropagation


In [11]:
# how many clusters were generated? 


In [12]:
# let's take a look at the clusters by printing the first 20
 

That's pretty neat! We get some clusters, and we can see how the words are put in the same groups. 

Is there anything kind of annoying? Too many one-word clusters? Clusters with too many words? Let's remove them to find some more insightful results.

In [13]:
def get_insightful_clusters(clusters, min_len=3, max_len=25):
    pass

get_insightful_clusters(clusters)[:20]

These clusters are okay, but there's definitely a lot that could be improved. 

Quiz: What would make these clusters better?
- more data
- longer convergence iter
- less weight on character parts

_Wait a second, what do you mean by better?_ 

### How do I compare two clustering results?

A metric exists to do this: the [silhouette score](https://en.wikipedia.org/wiki/Silhouette_(clustering)#:~:targetText=The%20silhouette%20value%20is%20a,poorly%20matched%20to%20neighboring%20clusters.). 

> The silhouette value is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation).

From [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score)
> The Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample. The Silhouette Coefficient for a sample is (b - a) / max(a, b). To clarify, b is the distance between a sample and the nearest cluster that the sample is not a part of. 

Sure was clear, eh?

Let's get a better definition of $a$ and $b$. 

![intra-cluster distance](figures/silhouette-a.png)

So for each point $i$ in a cluster, we define $a(i)$ to be the average distance between $i$ and every other point in the cluster. This is a measure of how well $i$ fits in its cluster. 

What kind of value (smaller or larger) for $a(i)$ indicates a good fit?

![mean nearest-cluster distance](figures/silhouette-b.png)

We find the next-best fitting (or _neighboring_) cluster for point $i$, and measure the average distance from $i$ to all points in that next-best fitting cluster. 

What does a high value for $b(i)$ indicate? A low one?

Now actually to the funciton for the silhouette score...

![silhouette-score-def](figures/silhouette-def.png)

Which can also be written as a piecewise function that's easier to analyze

![silhouette-rewritten](figures/silhouette-rewritten.png)

How do $a(i)$ and $b(i)$ compare in each of these situations? 

Does this make the silhouette score bounded or unbounded?

Is a higher or lower silhouette score better?

Let's get a visualization of what's really going on.

![silhouette visualized](figures/silhouette-visualized.gif)

Since we care about more than a single point in the clustering results, we average over all points to get teh silhouete score for a data set.

![Silhouette scores](figures/silhouette_analysis.png)


In [15]:
from sklearn.metrics import silhouette_score
# sklearn computes the mean silhouette score for all samples
silhouette_score(vectors, af.labels_)

0.03934189

Let's compare this with the unphrased model

In [14]:
# load in the unphrased model
unphrased_model_path = os.path.join('models', 'got_ft.model')


In [15]:
# preprocess and cluster


In [16]:
# get the silhouette score


## Which model was better?

Objectively: Which model had the better silhouette score? 

Subjectively: Let's check out the clusters too.

In [17]:
# look at the first 20 clusters from the unphrased model


Sure enough, seems like worse clusters without the phrased data. 

## Labeling Clusters Using a Hierarchy

Powerpoint presentation [here](https://callminer-my.sharepoint.com/:p:/p/kirsten_stallings/Ef_H9hSc15FPnujGQDGVdp4B6zCQGJZSpjgCs_ZfSBChxQ?e=d6zbhY). 

These clusters are insightful, but there's a lot of them to dig through. It's hard to know where to start. 

If each cluster had a _label_, we could sort on that label and get things closer together, focusing on the clusters we want to refine first. 

In order to label these clusters, we need a resource to know which words tend to get this label. This is the _hierarchy_, a tree of terms. 

Each node cruciall has
- a name or label, like `root-feeling-negative_sentiment-angry`

and each node _might_ have 
- children -  child of `root-feeling-negative_sentiment-angry` might be `root-feeling-negative_sentiment-angry-irate`
- parents - the parent of `root-feeling-negative_sentiment-angry` is `root-feeling-negative_sentiment`
- words that are associated with this label, like `upset, frustrated, in_denial, angry`

Each child of that node contains concepts that are more specific. 

For instance, 
> dog -> herders -> shetland sheepdog

Given a cluster and a word embedding model, the _labeler_ moves through the hierarchy to determine the best label for this unseen cluster.

![hiearchy and labeler](figures/hierarchy-and-labeler.png)

## The Hierarchy

Here's an example from the hierarchy we created by using clusters from call center data. 

![hierarchy expanding](figures/hierarchy-expanding.gif)

Each of these children of root holds a lot of significance in the call center space. That's the first distinction we make on a word/phrase!

These concepts are crucially never client or product specific. Since this will be used across many different clients, we don't want the product name from one to influence another incorrectly. 

## The Labeler

The labeler moves throught the hierarchy to find the best fitting name for the unseen cluster generated above in the clustering step. 

First, we find the centroid of the unseen cluster. We average all the word vectors to get the _centroid_, a vector that represents this cluster in space. 

The labeler is looking to find the closest centroid in the hierarchy to the centroid of the cluster we are currrently looking at. We look at two centroids:

1. The vectors for the words in the node and it's subtree
2. The vectors for only the words in the node.

We use the client's FastText model to calculate the centroids of the hierarchy nodes, effectively projecting the hierarchy into the client's space. 

Quiz: _Why is it important to use the client's model here?_

We take all the word vectors in the subtree rooted at the current node and average them to get the centroid. The serves as the middle location for this node in the the client's vector space. For each child of root, we have a centroid. 

We also compare with the centroid based on the words at this node _only_. If either of the two of those centroids is closer than the previously-considered-closest centroid, we move to this node in the tree and repeat with its children. 

When the closest centroid is found (none of the children's centroids are closer than the current node's), we label this cluster with the name of this node in the hierarchy. 

![moving through the hierarchy](figures/moving-through-the-hierarchy.gif)


In [20]:
# we can't show you any code for this - we'd have to build a whole new GOT hierarchy

Here are some examples of the hierarchy and labeler performing on real call center clusters:

![real life examples](figures/labeled-clusters.png)

# Summary
- what is clustering?
- clustering algorithms
    - k-means
    - hierarchical clustering
        - affinity propagation
- clustering word embeddings
    - how to get insight from clusters of word embeddings
- labeling clusters using a hierarchy